In [1]:
import geopandas as gpd
import pandas as pd
pd.set_option('display.max_columns', 80)
from shapely.geometry import Point, LineString
from geopandas import GeoDataFrame
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import datetime

In [2]:
# read in classified deeds file
rtt_classifiedRead = gpd.read_file("data/rtt_investors_classified.geojson")



DriverError: data/rtt_investors_classified.geojson: No such file or directory

In [ ]:
# read in grantor classified deeds file
rtt_grantorRead = gpd.read_file("data/rtt_grantor-investors_classified.geojson")

rtt_grantorCleaned = rtt_grantorRead.filter(['objectid','grantor_inv','grantor_occ'])

rtt_classified = rtt_classifiedRead.merge(
    rtt_grantorCleaned, left_on="objectid", right_on="objectid",how="left")


rtt_classified

In [ ]:
# Specify coordinate reference system
rtt_classified.to_crs='epsg:2272'

#rtt_classified.head()

In [ ]:
city_limits = gpd.read_file("data/02_flip-investigations_data/Map_Base/PHI_City_Limits.geojson")


In [ ]:
city_limits.crs 


In [ ]:
PHI_city_limits = city_limits.to_crs(2272)

In [ ]:
# filter for grantees
rtt_grantees = rtt_classified.loc[(rtt_classified['inv'] == 1) | (rtt_classified['occ'] == 1)] 
#rtt_investors.to_file("data/02_flip-investigations_data/rtt_investorsOnly.geojson", driver='GeoJSON')

# filter for grantors

rtt_grantors = rtt_classified.loc[(rtt_classified['grantor_inv'] == 1) | (rtt_classified['grantor_occ'] == 1)] 
#rtt_occ.to_file("data/02_flip-investigations_data/rtt_occupantsOnly.geojson", driver='GeoJSON')



rtt_grantors


In [ ]:
#GRANTEES

#CHECK DATETIME AND CLEAN
rtt_grantees['display_date_x'] = pd.to_datetime(rtt_grantees['display_date_x'])

rtt_grantees_sorted = rtt_grantees.sort_values(by=['street_address_x','display_date_x'])
grantee_resetIndex = rtt_grantees_sorted.reset_index()
grantee_cleanDF = grantee_resetIndex.filter(['id','C_DIG1DESC','C_DIG2DESC','C_DIG3DESC','OBJECTID','YEAR','VACBLDG','adjusted_cash_consideration','adjusted_total_consideration','document_id','document_type','fair_market_value','grantees_x_x','grantors_x','inv','lat','lng','matched_regmap','multi_invest','occ','owner_occup','property_count','display_date_x','recording_date','reg_map_id','sort_year_x','street_address_x','total_consideration','ward','unit_num','zip_code','geometry'])



#GROUP AND ROLLING DIFFERENCE

grantee_beg = (grantee_cleanDF.street_address_x == grantee_cleanDF.street_address_x.shift(1))                                                  
grantee_cleanDF['deedDiff2'] = (grantee_cleanDF['display_date_x'] - grantee_cleanDF['display_date_x'].shift(1))/np.timedelta64(1,'D')
grantee_cleanDF['deedDiff'] = grantee_cleanDF['deedDiff2'].where(grantee_beg,0).ffill()

#FILTER OUT DUPLICATES - filter out rows that have same address + display date


rtt_deedDiff_grantee = grantee_cleanDF.drop_duplicates(
    subset = ['street_address_x','display_date_x'],
    keep = 'last').reset_index(drop=True)


#rtt_deedDiff_grantee.to_file("data/02_flip-investigations_data/rtt_deedDiff_grantee.geojson", driver='GeoJSON')

# FILTER FOR FLIPS

#first take out 0s

grantee_no0 = rtt_deedDiff_grantee.loc[rtt_deedDiff_grantee['deedDiff'] != 0.0]
#then filter for less than 1.5 years (575 days)
rtt_Flips_grantee = grantee_no0.loc[grantee_no0['deedDiff'] <= 575]

#rtt_Flips_grantee.to_file("data/02_flip-investigations_data/rtt_Flips_grantee.geojson", driver='GeoJSON')


In [ ]:
#GRANTORS

#CHECK DATETIME AND CLEAN
rtt_grantors['display_date_x'] = pd.to_datetime(rtt_grantors['display_date_x'])


rtt_grantors_sorted = rtt_grantors.sort_values(by=['street_address_x','display_date_x'])
grantors_resetIndex = rtt_grantors_sorted.reset_index()
grantors_cleanDF = grantors_resetIndex.filter(['id','C_DIG1DESC','C_DIG2DESC','C_DIG3DESC','OBJECTID','YEAR','VACBLDG','adjusted_cash_consideration','adjusted_total_consideration','document_id','document_type','fair_market_value','grantees_x_x','grantors_x','grantor_inv','lat','lng','matched_regmap','multi_invest','grantor_occ','inv','occ','owner_occup','property_count','display_date_x','recording_date','reg_map_id','sort_year_x','street_address_x','total_consideration','ward','unit_num','zip_code','geometry'])


#GROUP AND ROLLING DIFFERENCE

grantors_beg = (grantors_cleanDF.street_address_x == grantors_cleanDF.street_address_x.shift(1))                                                  
grantors_cleanDF['deedDiff2'] = (grantors_cleanDF['display_date_x'] - grantors_cleanDF['display_date_x'].shift(1))/np.timedelta64(1,'D')
grantors_cleanDF['deedDiff'] = grantors_cleanDF['deedDiff2'].where(grantors_beg,0).ffill()


#FILTER OUT DUPLICATES - filter out rows that have same address + display date
                 
rtt_deedDiff_grantors = grantors_cleanDF.drop_duplicates(
    subset = ['street_address_x','display_date_x'],
    keep = 'last').reset_index(drop=True)

#rtt_deedDiff_grantors.to_file("data/02_flip-investigations_data/rtt_deedDiff_grantors.geojson", driver='GeoJSON')

# FILTER FOR FLIPS 

#first take out 0s

grantors_no0 = rtt_deedDiff_grantors.loc[rtt_deedDiff_grantors['deedDiff'] != 0.0]
#then filter for less than 1.5 years (575 days)
rtt_Flips_grantors = grantors_no0.loc[grantors_no0['deedDiff'] <= 575]


#rtt_Flips_grantors.to_file("data/02_flip-investigations_data/rtt_Flips_grantors.geojson", driver='GeoJSON')

#rtt_Flips_grantors.head(20)[['deedDiff','display_date_x','grantors_x','grantees_x_x','street_address_x']]
#rtt_Flips_grantors





In [ ]:
rtt_Flips_grantors.size


In [ ]:
rtt_Flips_grantors_inv = rtt_Flips_grantors.loc[rtt_Flips_grantors['grantor_inv']==1]
rtt_Flips_grantors_occ = rtt_Flips_grantors.loc[rtt_Flips_grantors['grantor_occ']==1]
rtt_Flips_grantors_invToOcc = rtt_Flips_grantors.loc[~(rtt_Flips_grantors['grantor_inv']==1) & (rtt_Flips_grantors['occ']==1)]


#rtt_Flips_grantors_inv.to_file("data/02_flip-investigations_data/rtt_Flips_grantors_inv.geojson", driver='GeoJSON')
#rtt_Flips_grantors_invToOcc.to_file("data/02_flip-investigations_data/rtt_Flips_grantors_invToOcc.geojson", driver='GeoJSON')
                                                     
rtt_Flips_grantors_inv.head(20)[['deedDiff','display_date_x','grantors_x','grantees_x_x','street_address_x']]                                                
                                                
#rtt_owner_check = rtt_investors.loc[rtt_investors['owner_occup'] == 1] 

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

ax.set_box_aspect(aspect=1)

PHI_city_limits.plot(ax=ax, color='white', edgecolor='black')


#rtt_classified.plot(ax=ax, color='blue')

#rtt_investors.plot(ax=ax, color = 'blue')
#rtt_Flips_grantee.plot(ax=ax, color = 'purple')


rtt_Flips_grantors.plot(ax=ax, color = 'red')




In [ ]:
# All grantors plot
#total_bar = total.reset_index()
#f, (ax1) = plt.subplots(1,1,figsize=(40, 10))
#sns.set(style="white")

#plt.plot(rtt_Flips_grantors['sort_year_x'], marker='o', color = 'black')
#plt.plot(total_bar['year'],total_bar['investor_all'], marker='o', color = 'black')
#plt.plot(total_bar['year'],total_bar['inv'], marker='o', color = 'black')

In [ ]:
#Grantor investor bar graph

f, (ax1) = plt.subplots(1,1,figsize=(10, 10))
# sns.set(style="white")

plt.bar(rtt_Flips_grantors['display_date_x'],rtt_Flips_grantors['fair_market_value'])

#plt.scatter(rtt_Flips_grantors['display_date_x'],rtt_Flips_grantors['fair_market_value'])


In [ ]:
# Calculations

print(rtt_Flips_grantors.size/rtt_classified.size)
print(rtt_Flips_grantors_inv.size/rtt_Flips_grantors.size)
print(rtt_Flips_grantors_invToOcc.size/rtt_Flips_grantors_inv.size)

In [ ]:
# All Flips Histogram
plt.title(label="All Flips")
plt.hist(rtt_Flips_grantors['display_date_x'])


#plt.savefig("data/02_flip-investigations_data/Hist_All-Flips.png")


In [ ]:
# All Flips (grantor vs. grantee) Histogram
plt.title(label="All Flips")
plt.hist([rtt_Flips_grantors['display_date_x'],rtt_Flips_grantee['display_date_x']],stacked=False, density=True)


#plt.savefig("data/02_flip-investigations_data/Hist_All-Flips_grantorxgrantee.png")

In [ ]:
# Investor Flips Histogram
plt.title(label="Flips by Investor")
plt.hist(rtt_Flips_grantors_inv['display_date_x'])


#plt.savefig("data/02_flip-investigations_data/Hist_Inv-Flips.png")

In [ ]:
# NON - Investor Flips Histogram
plt.title(label="Flips by Non - Investor")
plt.hist(rtt_Flips_grantors_occ['display_date_x'])


#plt.savefig("data/02_flip-investigations_data/Hist_Occ-Flips.png")

In [ ]:
# Investor to Occupant Flips Histogram
plt.title(label="Flips by Investor bought by Occupant")
plt.hist(rtt_Flips_grantors_invToOcc['display_date_x'])


plt.savefig("data/02_flip-investigations_data/Hist_InvToOcc-Flips.png")

In [ ]:
# Investor vs. Investor to Occupant Flips Histogram
plt.title(label="Flips by Investor vs. by Occupant")
plt.hist([rtt_Flips_grantors_inv['display_date_x'],rtt_Flips_grantors_occ['display_date_x']], stacked=False, density=True)


#plt.savefig("data/02_flip-investigations_data/Hist_Inv+Occ-Flips.png")


In [ ]:
# Grantor Flips Scatter

plt.title(label="Grantor Flips x Price")
plt.scatter(rtt_Flips_grantors['display_date_x'],rtt_Flips_grantors['deedDiff'])
#plt.savefig("data/02_flip-investigations_data/Scat_Owner-Flips.png")





In [ ]:
#grantor Flips Violin Plot


plt.title(label="Investor Flips x Price")
plt.violinplot(rtt_Flips_grantors['display_date_x'],rtt_Flips_grantors['deedDiff'])



In [ ]:
#rtt_deedDiff_grantee.head(30)[['deedDiff','display_date_x','street_address_x']]

#rtt_Flips_grantors.head(10)[['deedDiff','display_date_x','grantors_x','grantees_x_x','street_address_x']]

check_all = rtt_classified.loc[(rtt_classified['street_address_x'] == '1000 S COLORADO ST')]

check_all['display_date_x'] = pd.to_datetime(check_all['display_date_x'])
check_all_sorted = check_all.sort_values(by=['street_address_x','display_date_x'])

check_all_sorted.head(20)[['display_date_x','grantors_x','grantees_x_x','street_address_x']]




In [ ]:
#check_deedDiff = grantors_cleanDF.loc[(grantors_cleanDF['street_address_x'] == '1000 S COLORADO ST')]


check_deedDiff = rtt_deedDiff_grantors.loc[(rtt_deedDiff_grantors['street_address_x'] == '1000 S COLORADO ST')]





check_deedDiff.head(20)[['deedDiff','display_date_x','grantors_x','grantees_x_x','street_address_x']]

In [ ]:
check_Flips = rtt_Flips_grantors.loc[(rtt_Flips_grantors['street_address_x'] == '1000 S COLORADO ST')]


check_Flips.head(50)[['deedDiff','display_date_x','grantors_x','grantees_x_x','street_address_x']]


In [ ]:
#GRAVEYARD

# check for owner_occ - check is good
#rtt_owner_check = rtt_investors.loc[rtt_investors['owner_occup'] == 1] 
#rtt_owner_check.shape

#_grouped = df.groupby("street_address_x").rolling(window=1).Amount.count()

#rtt_investors_dateindex['flip_time'] = rtt_investors_dateindex.groupby(['street_address_x']).document_date.astype('timedelta64[D]')

#cleanDF['deedDiff'] = cleanDF.groupby(['street_address_x','receipt_date']).receipt_date.apply(lambda x: x - x.iloc[0])
#cleanDF['deedDiff'] /= np.timedelta64 (1, 'D')

#grantors_cleanDF['deedDiff'] = grantors_cleanDF.groupby(['street_address_x','display_date_x']).display_date_x.apply(lambda x: x - x.iloc[0])
#grantors_cleanDF['deedDiff'] /= np.timedelta64 (1, 'D')


#grantors_cleanDF['deedDiff'] = (grantors_cleanDF['display_date_x'] - grantors_cleanDF['display_date_x'].where(grantors_beg).ffill())/np.timedelta64(1,'D')

#grantors_cleanDF['deedDiff']

#grantors_cleanDF['deedDiff'] = grantors_cleanDF.groupby(['display_date_x']).display_date_x.apply(lambda x: x - x.iloc[0]).apply(lambda x:x / np.timedelta64(1, 'D'))
#grantors_cleanDF['deedDiff'] = grantors_cleanDF.groupby('street_address_x')['display_date_x'].diff().fillna(pd.Timedelta(seconds=0))




#rtt_investors.crs = 'epsg:4326'
#PHI_rtt_investors = rtt_investors.to_crs('epsg:2272')

#OCCUPANTS

'''  # FILTER FOR ADDRESS / ZIP THAT APPEARS MOST TIMES

topzip = rtt_classified['zip_code'].value_counts() 
topaddress = rtt_classified['street_address_x'].value_counts() 

#topaddress.head(10)
#rtt_investors_topaddress.to_file("data/rtt_investors_topaddress.geojson", driver ='GeoJSON')

topaddress_1000 = topaddress.iloc[:1000]
addresses=topaddress_1000.index
a_list=list(addresses)
#list2 = ['12 HAMILTON CIR,' '3817 POPLAR ST']

#is_top = rtt_test['street_address_x'].isin(a_list)
is_top = rtt_classified['street_address_x'] == '12 HAMILTON CIR'
#is_top = rtt_test['zip_code'] == 19134
rtt_top = rtt_classified[is_top]
#rtt_top['property_count'].mean()
rtt_top['receipt_date'] = pd.to_datetime(rtt_top.receipt_date)
rtt_top.sort_values(by=['receipt_date'])
rtt_top.shape



#rtt_top.to_html('temp.html') '''

'''
#CHECK DATETIME AND CLEAN
rtt_occ['receipt_date'] = pd.to_datetime(rtt_occ['receipt_date'])

rtt_occ_sorted = rtt_occ.sort_values(by=['street_address_x','receipt_date'])
occ_resetIndex = rtt_occ_sorted.reset_index()
occ_cleanDF = occ_resetIndex.filter(['id','C_DIG1DESC','C_DIG2DESC','C_DIG3DESC','OBJECTID','YEAR','VACBLDG','adjusted_cash_consideration','adjusted_total_consideration','display_date_x','document_date','document_id','document_type','fair_market_value','grantees_x_x','grantors_x','inv','lat','lng','matched_regmap','multi_invest','occ','owner_occup','property_count','receipt_date','recording_date','reg_map_id','sort_year_x','street_address_x','total_consideration','ward','unit_num','zip_code','geometry'])


#GROUP AND ROLLING DIFFERENCE


occ_beg = (occ_cleanDF.street_address_x != occ_cleanDF.street_address_x.shift(1)) 
occ_cleanDF['deedDiff'] = (occ_cleanDF['receipt_date'] - occ_cleanDF['receipt_date'].where(occ_beg).ffill())/np.timedelta64(1,'D')
rtt_deedDiff_occ = occ_cleanDF

#rtt_deedDiff_occ.to_file("data/02_flip-investigations_data/rtt_deedDiff_occ.geojson", driver='GeoJSON')

# FILTER FOR FLIPS

#first take out 0s
occ_no0 = rtt_deedDiff_occ.loc[rtt_deedDiff_occ['deedDiff'] != 0.0]
#then filter for less than 1.5 years (575 days)
rtt_Flips_occ = occ_no0.loc[occ_no0['deedDiff'] <= 575]

#rtt_Flips_occ.to_file("data/02_flip-investigations_data/rtt_Flips_occ.geojson", driver='GeoJSON')
'''

